In [1]:
import numpy as np
import sys
import os

In [2]:
module_path = os.path.abspath(os.path.join('..')) # Path to root folder
if module_path not in sys.path:
    sys.path.append(module_path + "/scripts") # define scripts path

from ipynb_func import *

In [3]:
NUM = 1 # Number of data parquets to use

assert NUM >= 1 and NUM <= 10, "NUM value must be in range [1, 10]"
paths = [module_path + f"/data/pikabu/tag_processed/{i}_tag_processed.parquet" for i in range(NUM)] # Making list of roots to merge 

data = merge_dataset(paths)

In [4]:
data.head(5)

,text_markdown,tags
8,"[сломаться, китайский, телевизор, сломаться, к...",[Юмор]
10,"[вспоминать, успешный, операция, британский, м...",[История]
11,"[жаловаться, трудиться, помощник, руководитель...","[Еда, Текст]"
15,"[добрый, сутки, господин, дама, подсказывать, ...","[Игры, Текст]"
23,"[негр, ходить, больница, читать, зад, больной,...",[Текст]


In [5]:
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [6]:
data_text = [t.tolist() for t in data.text_markdown]
dict = Dictionary(data_text)
corpus = [dict.doc2bow(text) for text in data_text]

#[[(dict[i], freq) for i, freq in line] for line in corpus[:1]]

In [7]:
tags_list = getwordlist(data.tags)
tags_list = getworddict(tags_list)
num_topics = len(tags_list)
print('Number of topics to predict:', num_topics)

Number of topics to predict: 48


In [8]:
np.random.seed(4228)

lda_cfg = {'num_topics': num_topics,
           'epochs': 10,
           'chunksize': 100,
           'update_every': 10}

In [9]:
LDA = LdaModel(corpus=corpus,
               id2word=dict,
               num_topics=lda_cfg['num_topics'],
               passes=lda_cfg['epochs'],
               chunksize=lda_cfg['chunksize'],
               update_every=lda_cfg['update_every'],
               alpha='auto',
               eta='auto')

In [10]:
print('Perplexity of a model:', LDA.log_perplexity(corpus))

Perplexity of a model: -8.217921785430276


In [11]:
coher = CoherenceModel(model=LDA, texts=data_text, dictionary=dict, coherence='c_v')
coher_lda = coher.get_coherence()
print('Coherence of a model:', coher_lda)

Coherence of a model: 0.4546552583663923


In [25]:
LDA.show_topics(num_topics=10, formatted=True)

[(7,
  '0.043*"салон" + 0.026*"авария" + 0.023*"водитель" + 0.022*"оружие" + 0.021*"команда" + 0.019*"автомобиль" + 0.018*"останавливать" + 0.018*"тренер" + 0.017*"попадать" + 0.016*"пистолет"'),
 (44,
  '0.081*"германия" + 0.040*"польша" + 0.033*"немец" + 0.028*"франция" + 0.027*"гитлер" + 0.026*"европа" + 0.025*"парк" + 0.024*"немецкий" + 0.024*"польский" + 0.020*"ссср"'),
 (22,
  '0.070*"станция" + 0.032*"строительство" + 0.029*"перевод" + 0.028*"площадь" + 0.020*"напряжение" + 0.020*"мороженое" + 0.018*"ток" + 0.017*"ленин" + 0.013*"ответ" + 0.012*"минск"'),
 (37,
  '0.077*"пакет" + 0.054*"гараж" + 0.033*"пластиковый" + 0.032*"код" + 0.028*"долг" + 0.023*"горячий" + 0.022*"марс" + 0.019*"копейка" + 0.017*"резать" + 0.017*"отправка"'),
 (46,
  '0.093*"сергей" + 0.040*"инспектор" + 0.032*"телевизор" + 0.024*"конкурс" + 0.022*"иванов" + 0.022*"зуб" + 0.018*"лук" + 0.016*"чистить" + 0.015*"мед" + 0.015*"яблоко"'),
 (28,
  '0.007*"система" + 0.007*"сша" + 0.006*"доллар" + 0.005*"миллион

In [13]:
top = LDA.top_topics(corpus=corpus)
coh = []
for topic in top:
    for word in topic[0]:
        coh.append(word[0])
coh = np.mean(coh)

print('Mean coherence of a words: ', coh)

Mean coherence of a words:  0.011597135


In [14]:
THRESHOLD = 0.15

corpus_topics = []
for sentence in corpus:
    topics = LDA.get_document_topics(sentence)
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    topics_for_sentence = []
    for i in range(len(topics)):
        if topics[i][1] > THRESHOLD:
            topics_for_sentence.append(topics[i][0])
    corpus_topics.append(topics_for_sentence)

data['corpus_topics'] = corpus_topics

In [15]:
data.head(10)

,text_markdown,tags,corpus_topics
8,"[сломаться, китайский, телевизор, сломаться, к...",[Юмор],"[15, 10]"
10,"[вспоминать, успешный, операция, британский, м...",[История],"[21, 31]"
11,"[жаловаться, трудиться, помощник, руководитель...","[Еда, Текст]",[3]
15,"[добрый, сутки, господин, дама, подсказывать, ...","[Игры, Текст]",[29]
23,"[негр, ходить, больница, читать, зад, больной,...",[Текст],"[34, 7, 39]"
28,"[мужик, российский, али, смена, совесть, пород...",[Текст],"[3, 39]"
29,"[дополнение, пост, уровень, российский, наука,...",[СССР],"[19, 37, 28]"
33,"[центр, киев, взрыв, предварительный, данный, ...","[Новости, Украина]",[9]
37,"[ехать, девчонка, школа, оставаться, свободный...","[Юмор, Текст]","[10, 15]"
41,"[яблоко, проклятие, вспоминать, прошлый, век, ...",[Мат],"[46, 3]"


In [16]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(LDA, corpus, dict)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.126533  0.140537       1        1  10.011993
5     -0.116983 -0.041020       2        1   7.705269
28     0.219388  0.019843       3        1   6.257958
39    -0.190611 -0.046556       4        1   5.310751
1      0.231070 -0.021822       5        1   4.716059
19     0.098650  0.060457       6        1   3.574482
31     0.127981  0.137053       7        1   3.382918
40    -0.129447  0.033123       8        1   3.152518
17     0.011457  0.012642       9        1   3.076372
26     0.033952 -0.139221      10        1   2.803276
25     0.069556 -0.204540      11        1   2.782116
0     -0.185203  0.038317      12        1   2.661660
13    -0.071275 -0.193363      13        1   2.624983
34     0.006575 -0.010486      14        1   2.594812
3     -0.077746  0.016047      15        1   2.217170
14     0.228098 -0.099481      16        1   2.152145
45     0.069517 -0.221974      17        1   2.053335
32     0.094523  0.087765      18        1   1.894800
2     -0.202128  0.076515      19        1   1.816836
29     0.061514 -0.088887      20        1   1.802879
36     0.146960  0.162340      21        1   1.696127
27    -0.140729  0.058882      22        1   1.659876
24     0.197067  0.073495      23        1   1.463277
12    -0.038755  0.186120      24        1   1.460028
42    -0.162695  0.049380      25        1   1.401620
10    -0.060235 -0.132758      26        1   1.373865
41    -0.078583  0.034542      27        1   1.369641
15    -0.068097 -0.078639      28        1   1.251484
21     0.094547  0.077616      29        1   1.224499
20     0.013820  0.019502      30        1   1.179470
38    -0.121287  0.072681      31        1   1.133700
18     0.001146 -0.086176      32        1   1.124412
16     0.106143 -0.003472      33        1   1.121682
33    -0.154285 -0.074070      34        1   1.004833
35    -0.013154 -0.081110      35        1   0.846207
11     0.016649 -0.087303      36        1   0.814211
47     0.007921  0.075718      37        1   0.798451
43    -0.082509 -0.067169      38        1   0.793586
9     -0.043455 -0.042785      39        1   0.791863
4     -0.037188  0.057627      40        1   0.741175
23    -0.048920  0.077133      41        1   0.735872
30    -0.035701  0.077629      42        1   0.716970
6      0.028363 -0.050318      43        1   0.712098
44     0.153784  0.120860      44        1   0.574618
22     0.068433 -0.026311      45        1   0.430333
7      0.050289  0.018562      46        1   0.368846
46    -0.006542  0.022042      47        1   0.346141
37     0.054660 -0.008968      48        1   0.272783, topic_info=              Term         Freq        Total Category  logprob  loglift
1384      квартира  1267.000000  1267.000000  Default  30.0000  30.0000
494        девушка  1461.000000  1461.000000  Default  29.0000  29.0000
475           пост  1601.000000  1601.000000  Default  28.0000  28.0000
739           мама  1333.000000  1333.000000  Default  27.0000  27.0000
844         парень  1075.000000  1075.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
3454           чек    14.682270   101.377113  Topic48  -4.9722   3.9720
1844          воля    16.169744   188.079375  Topic48  -4.8757   3.4505
1246  использовать    20.358006   651.822435  Topic48  -4.6453   2.4380
1389    оплачивать    15.402719   287.465947  Topic48  -4.9243   2.9777
476   производство    14.216104   361.519044  Topic48  -5.0044   2.6683

[2588 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
5714     18  0.113279   and
5714     20  0.022656   and
5714     38  0.747640   and
5714     41  0.067967   and
5714     43  0.022656   and
...     ...       ...   ...
3183     30  0.009560  ящик
3183     33  0.019120  ящик
3183     36  0.478003  ящик
3183     40  0.109941  ящик
3183     43  0.0191

In [22]:
a = input('Are you sure, that you want to save the model? (Y/n)')
if a == 'Y':
    model_path = module_path + '/models/'
    LDA.save(model_path + 'lda_model_for_48_topics.model')